## Introduction

This recipe shows how to add the phase information into the EventLists of pulsar observations. This step is needed to perform the pulsar analysis with gammapy and should be the first step in the high level analysis. For the pulsar analysis we need two ingredients:

1. The time of arrivals (TOAs). These times should have very high precision due to the common fast periods of pulsars. Usually these times are already stored in the EventList tables. For the computation of the pulsar timing they have to be corrected to be referenced to the Solar System barycenter (SSB) because this system can nearly be regarded as an inertial frame with respect to the pulsar. Sometimes, (for rather historical reasons) the times are stored in specific files, called .tim files.


2. The model of rotation of the pulsar, also known as ephemeris, at the epoch of the observations. These ephemerides are stored in an specific format and saved as .par files and contain information of the periods, derivatives of the periods, coordinates, glitches, etc.

__For following the steps of this tutorial, we need the original EventLists from the DL3 files, and a model in .par format.__

The main software that will help us to make the barycentric corrections and the phase-folding to the model is the PINT library. For more information about how this package works, see the following link: https://nanograv-pint.readthedocs.io/en/latest/. 

## 0. Imports and dependencies

The dependencies that we will be using in this tutorial are essentials (astropy, matplotlib, numpy). We need also to install PINT for the phase computation (see https://nanograv-pint.readthedocs.io/en/latest/installation.html for more information).

The version of gammapy used in this tutorial is 1.0

In [ ]:
import gammapy
import astropy

print(f'Version of gammapy: {gammapy.__version__}')
print(f'Version of astropy: {astropy.__version__}')

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
import pint.models as models
import numpy as np
import os
from pathlib import Path

In [ ]:
from gammapy.utils.regions import SphericalCircleSkyRegion
from gammapy.data import DataStore, EventList, Observation, Observations
from gammapy.data import GTI
from astropy.coordinates import SkyCoord
from gammapy.makers import (
    SafeMaskMaker,
    PhaseBackgroundMaker,
    SpectrumDatasetMaker,
)
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.maps import Map, WcsGeom, MapAxis, RegionGeom

## 1. Reading DataStore

In [ ]:
# Define the directory containing the DL3 data
DL3_direc = "/Users/Jan-Lukas/Desktop/Uni/Promotion/gammapy/gammapy-data/magic/rad_max/data"
#DL3_direc='/Users/alvarom/Desktop/Doctorado/fermipulsar/gammapy-datasets/1.0/magic/rad_max/data/'

In [ ]:
#Read DataStore from a directory
total_datastore = DataStore.from_dir(DL3_direc)

In [ ]:
#Extract observations from desired obs_ids
obs_ids = None

if obs_ids is None:
    total_obs_list = total_datastore.obs_table["OBS_ID"].data
else:
    total_obs_list = obs_ids
    
observations = total_datastore.get_observations(total_obs_list, required_irf = "point-like")

## 2. Phase-folding with PINT for one observation

Let's extract the times from the observations:

In [ ]:
#Extract times from EventList
observation = observations[1]
times = observation.events.time

In [ ]:
times

Now we have the TOAs of the events in the system of the telescope. Please note that the actual precision of the times is higher than the diplayed output (and we really need this precision for the pulsar analysis!). In the next step, the timing in the SSB and the phase for each TOA has to be created. 

At this point we have two ways to proceed leading to the same results. One can choose between onr or another depending on their analysis case.

## 2.1 First way: creating a temporary .tim file

The first method is to create a .tim file to be used as the input of PINT. This step makes the analysis with PINT straight-forward, without needing to know the internal function of PINT. PINT uses this .tim file to create all the TOA objects and to do the needed corrections automatically.

### 2.1.1 Creating a basic .tim file with the TOAs

First we need to set a random name for the .tim file (in case you want to keep them) and the observatory name used by PINT (same as tempo2).

In [ ]:
timname = 'times.tim'
obs = 'magic'

We create the .tim file

In [ ]:
timelist = list(times.to_value('mjd','long'))

timFile = open(timname,'w+')
timFile.write('FORMAT 1 \n')
for i in range(0,len(timelist)):
        timFile.write(str(obs) + ' ' + '0.0 ' + str(times[i]) + ' 0.0 ' + str(obs) + ' \n')
timFile.close()

In [ ]:
!head times.tim

#### Some more details about .tim files

A .tim file is basically a text file containing five columns:
* Filename/Identifier: _telescope/observatory_
* Observing Frequency: `0.0`
* Arrival Time (MJD): _time_
* Uncertainty (µs): `0.0`
* Telescope Code: _telescope identifier_

The .tim files are still a relict from tempo2 (which was developed mainly for the radio astronomy), so there are parameters such as the observing frequency which don't really make sense for the gamma-ray astronomy. So the observing frequency and the uncertainty are usually set to 0 for gamma-ray astronomy. 

### 2.1.2. Setting and exploring model

In [ ]:
ephemeris_file = '/Users/alvarom/Desktop/Doctorado/fermipulsar/crab_ephemeris_model.par'

ephemeris_file = '/Users/Jan-Lukas/Desktop/Uni/Promotion/Pulsar_Analyse/pulsar_analysis_crab/parfiles/eph_0531+21_57479_57494_57509_J.par'

In [ ]:
%%time 
m, t = models.get_model_and_toas(ephemeris_file, timname, planets = True,usepickle = False)

PINT has already read the model and created the TOAs with all the corrections. Let's inspect the model, which follows the .par format:

In [ ]:
print(m)

The model contains information such as the pulsar name (PSR), the start (START) and end (FINISH) date for which this particular ephmeris is valid, the coordinates of the pulsar (RAJ, DECJ), and the pulsar frequency (F0) and its derivatives (F1, F2).

A model in PINT consists of two things: 

* The delay functions which specify all the delays to take into account when transforming the arrival time to the barycenter of the solar system.
* The Phase functions which describe the pulsar paying respect to the spindown model and the absolute phase definition.

__Remember that this model is only valid for a certain epoch so always check that the epoch of your observations are within the range of validity of the ephemeris.__

### 2.1.3 Compute phases using PINT

Let's obtain the barycentric times and the phases computed by PINT:

In [ ]:
%%time
barycent_toas = m.get_barycentric_toas(t)
phases = m.phase(t,abs_phase=True).frac
phases = np.array(phases)

In [ ]:
# create histogram of the phases
plt.hist(phases)
plt.show()

PINT compute the phases between -0.5 and 0.5. Normally, for pulsar analysis we work in the range [0,1], so we need to shift the negative phases.

In [ ]:
phases = np.where(phases < 0.0 , phases + 1.0 , phases)

In [ ]:
# create histogram of the shifted phases
plt.hist(phases)
plt.show()

Now we see that the phases range from 0 to 1.

### 2.1.4 Removing tim file (optional)

Since the original times are located in the EventList we can remove the .tim file created to be used by PINT.

In [ ]:
os.remove(str(os.getcwd()) + '/' + timname)

## 2.2 Second way: Using internal functions of PINT 

Actually, this should be the desirable way of proceeding but we need to understand better the functionalities of PINT. We do not need to create a .tim file, and we can create directly the TOAs object used in PINT by our own and perform the corrections step by step.

In [ ]:
from pint import toa
from pint import models
from pint.observatory.topo_obs import TopoObs
from pint.models import get_model

In [ ]:
model = get_model(ephemeris_file)

In [ ]:
%%time
include_bipm = True
include_gps = True

toa_list = list(toa.TOA(t, 4.5, obs='magic', ephem='DE421', include_bipm=str(include_bipm), 
                                include_gps=str(include_gps)) for t in times)
        
ts = toa.TOAs(toalist=toa_list)
ts.clock_corr_info={'include_bipm': str(include_bipm), 'include_gps': str(include_gps)}

ts.apply_clock_corrections(include_gps=include_gps,include_bipm=include_bipm)

ts.compute_TDBs(ephem='DE421')
ts.compute_posvels(ephem='DE421',planets=True)
        
phss = model.phase(ts,abs_phase=True)[1]
    
# shift phases to the interval (0,1]
phases_2 = np.where(phss < 0.0 , phss + 1.0 , phss)

## 3. Creating New Event List

In [ ]:
#Create new EventList with the phases
table = observation.events.table

In [ ]:
# show original table
table

In [ ]:
# add phases to table
table['PHASE'] = phases.astype('float64')
table.sort('TIME')

In [ ]:
# show table with phases
table

As you can see now the column 'PHASE' is added to the table.

## 4. Save new Event List

In [ ]:
# create new event list and add it to observation object
new_event_list = EventList(table)
observation._events = new_event_list

In [ ]:
# define output directory and filename
output_directory = './pulsar_data/'  
filename = f'dl3_pulsar_{observation.obs_id:04d}.fits'
file_path = output_directory + filename

Path(output_directory).mkdir(parents=True, exist_ok=True)

In [ ]:
# save the observation object in the specified file_path
print('Writing outputfile in '+str(file_path))
observation.events.write(filename = file_path, gti = observation.gti, overwrite=True)

## 5. Reading the new EventList files with the original DataStore information

-------------

## Making a phaseogram with basic statistics

In [ ]:
# define on and off region within the phase space
off_phase_range = (0.6, 0.9)
on_phase_range = (0.35, 0.45)

In [ ]:
#define source position and on radius
pos_target = SkyCoord(ra=83.63 * u.deg, dec=22.01 * u.deg, frame="icrs")          
on_radius = 0.2*u.deg

on_region = SphericalCircleSkyRegion(pos_target, on_radius)

In [ ]:
true_energy_axis = MapAxis.from_energy_bounds( 0.003, 10, 100, unit="TeV", name="energy_true")
reco_energy_axis = MapAxis.from_energy_bounds(0.01, 10, 30, unit="TeV", name="energy")


geom=RegionGeom.create(region = on_region, axes=[reco_energy_axis])
    
#Define empty Dataset
dataset_empty = SpectrumDataset.create(geom=geom,energy_axis_true=true_energy_axis)

#Define dataset maker
dataset_maker = SpectrumDatasetMaker(containment_correction=False, selection=["counts", "exposure", "edisp"], use_region_center=True)  

# Define background maker with the phase regions
phase_bkg_maker = PhaseBackgroundMaker(on_phase=on_phase_range, off_phase=off_phase_range)

dataset = dataset_maker.run(dataset_empty, observation)
dataset_on_off = phase_bkg_maker.run(dataset, observation)

In [ ]:
stats_info = dataset_on_off.info_dict()

In [ ]:
stats_info

## Draw the histogram

In [ ]:
phases_on_region = observation.events.select_region(on_region).table['PHASE']

In [ ]:
plt.figure(figsize=(15,6))
plt.title('Phaseogram of the sample',fontsize=20)
plt.hist(phases_on_region,bins=20, edgecolor='blue',facecolor='xkcd:sky blue',histtype='stepfilled')
plt.hist(phases_on_region+1,bins=20, edgecolor='blue',facecolor='xkcd:sky blue',histtype='stepfilled')
plt.fill_between(np.linspace(on_phase_range[0],on_phase_range[1]),0,650,color='C3',alpha=0.4)
plt.fill_between(np.linspace(on_phase_range[0],on_phase_range[1])+1,0,650,color='C3',alpha=0.4, label='ON region')
plt.fill_between(np.linspace(off_phase_range[0],off_phase_range[1]),0,650,color='k',alpha=0.3, hatch='/', label='OFF region')
plt.fill_between(np.linspace(off_phase_range[0],off_phase_range[1])+1,0,650,color='k',alpha=0.3, hatch='/')
plt.xlabel('Phase')
plt.ylabel('Counts')
plt.text(1.6,23,'Stats: \n Significance:{0:.2f}$\sigma$ \n N(ON):{1:.0f} \n N(off):{2:.0f} \n Nex:{3:.0f} \n alpha:{4:.2f}'.format(stats_info['sqrt_ts'], stats_info['counts'],stats_info['counts_off'],stats_info['excess'],stats_info['alpha']),
         fontsize=16,color='black',bbox=dict(facecolor='white', edgecolor='black'))
plt.text(-0.05,55,' Nentries:{0:.0f} \n Tobs:{1:.2f}'.format(len(table),stats_info['livetime'].to(u.Unit('h'))),
        fontsize=16,weight='bold',color='black',bbox=dict(facecolor='white', edgecolor='black'))
plt.ylim(20,60)
plt.legend(fontsize=15)
plt.show()

You can see the phasogram for your pulsar analysis. Please note that in this case we don't see peaks where we would expect them because the used datased is too small. 